# Tutorial 3: Learning

In [1]:
import libspn as spn
import tensorflow as tf

### Building a Test Graph with Random Weights
So, what does generate_ivs do? Basically it connects an IV to a sum node (it is a method defined in `SumNode`). Can be used for a latent variable interpretation.
```python
def generate_ivs(self, feed=None, name=None):
    """Generate an IVs node matching this sum node and connect it to
    this sum.

    IVs should be generated once all inputs are added to this node,
    otherwise the number of IVs will be incorrect.

    Args:
        feed (Tensor): See :class:`~libspn.IVs`.
        name (str): Name of the IVs node. If ``None`` use the name of the
                    sum + ``_IVs``.

    Return:
        IVs: Generated IVs node.
    """
    if not self._values:
        raise StructureError("%s is missing input values" % self)
    if name is None:
        name = self._name + "_IVs"
    # Count all input values
    num_values = sum(len(v.indices) if v.indices is not None
                     else v.node.get_out_size()
                     for v in self._values)
    ivs = IVs(feed=feed, num_vars=1, num_vals=num_values, name=name)
    self.set_ivs(ivs)
    return ivs
```

In [3]:
iv_x = spn.IVs(num_vars=2, num_vals=2, name="iv_x")
sum_11 = spn.Sum((iv_x, [0,1]), name="sum_11")
sum_12 = spn.Sum((iv_x, [0,1]), name="sum_12")
sum_21 = spn.Sum((iv_x, [2,3]), name="sum_21")
sum_22 = spn.Sum((iv_x, [2,3]), name="sum_22")
prod_1 = spn.Product(sum_11, sum_21, name="prod_1")
prod_2 = spn.Product(sum_11, sum_22, name="prod_2")
prod_3 = spn.Product(sum_12, sum_22, name="prod_3")
root = spn.Sum(prod_1, prod_2, prod_3, name="root")
# Some latent variable
iv_y = root.generate_ivs(name="iv_y")
# initialize weights randomly
spn.generate_weights(root, init_value=spn.ValueType.RANDOM_UNIFORM(0, 1))

### Visualizing the SPN Graph

In [4]:
spn.display_spn_graph(root)

### Specify Training Data

In [5]:
iv_x_arr=[
    [0,0], # in two cases x1 = x2 = 0 
    [0,0],
    [1,1], # in three cases x1 = x2 = 1
    [1,1],
    [1,1],
    [0,1], # in three cases x1 = 0 and x2 = 1
    [0,1],
    [0,1]
]
# no evidence for iv_y
iv_y_arr=[[-1]] * len(iv_x_arr)

### Add Learning Ops

In [6]:
# Build initialization op
init_weights = spn.initialize_weights(root)

# Build learning op
learning = spn.EMLearning(root, initial_accum_value=2)
init_learning = learning.reset_accumulators()
accumulate_updates = learning.accumulate_updates()
update_spn = learning.update_spn()
likelihood = tf.reduce_mean(learning.value.values[root])

Let's look at `EMLearning`:
```python
class EMLearning():
    """Assembles TF operations performing EM learning of an SPN.

    Args:
        mpe_path (MPEPath): Pre-computed MPE_path.
        value_inference_type (InferenceType): The inference type used during the
            upwards pass through the SPN. Ignored if ``mpe_path`` is given.
        log (bool): If ``True``, calculate the value in the log space. Ignored
                    if ``mpe_path`` is given.
    """

    ParamNode = namedtuple("ParamNode", ["node", "name_scope", "accum"])

    def __init__(self, root, mpe_path=None, log=True, value_inference_type=None,
                 additive_smoothing=None, add_random=None, initial_accum_value=None,
                 use_unweighted=False):
        self._root = root
        self._additive_smoothing = additive_smoothing
        self._initial_accum_value = initial_accum_value
        # Create internal MPE path generator
        if mpe_path is None:
            """ In the example above this is not given, so MPEPath is built here """
            
            self._mpe_path = MPEPath(log=log,
                                     value_inference_type=value_inference_type,
                                     add_random=add_random, use_unweighted=use_unweighted)
        else:
            self._mpe_path = mpe_path
        # Create a name scope
        with tf.name_scope("EMLearning") as self._name_scope:
            pass
        # Create accumulators
        self._create_accumulators()

```
The initialization of `MPEPath` is given here:
```python
class MPEPath:
    """Assembles TF operations computing the branch counts for the MPE downward
    path through the SPN. It computes the number of times each branch was
    traveled by a complete subcircuit determined by the MPE value of the latent
    variables in the model.

    Args:
        value (Value or LogValue): Pre-computed SPN values.
        value_inference_type (InferenceType): The inference type used during the
            upwards pass through the SPN. Ignored if ``value`` is given.
        log (bool): If ``True``, calculate the value in the log space. Ignored
                    if ``value`` is given.
    """

    def __init__(self, value=None, value_inference_type=None, log=True, add_random=None,
                 use_unweighted=False):
        self._counts = {}
        self._log = log
        self._add_random = add_random
        self._use_unweighted = use_unweighted
        # Create internal value generator
        if value is None:
            if log:
                """ Log value is used by default """
                self._value = LogValue(value_inference_type)
            else:
                self._value = Value(value_inference_type)
        else:
            self._value = value
```

Final call in constructor of EMLearning is `_create_accumulators()`
```python
def _create_accumulators(self):
    def fun(node):
        # Only cares about parameter nodes
        if node.is_param:
            with tf.name_scope(node.name) as scope:
                if self._initial_accum_value is not None:
                    # If there already is an initial accumulator value
                    
                    # Create a variable containing the initial accumulator value everywhere
                    # Add it to EM collection
                    accum = tf.Variable(tf.ones_like(node.variable,
                                                     dtype=conf.dtype) *
                                        self._initial_accum_value,
                                        dtype=conf.dtype,
                                        collections=['em_accumulators'])
                    
                else:
                    # Otherwise, use zero as accumulator init
                    accum = tf.Variable(tf.zeros_like(node.variable,
                                                      dtype=conf.dtype),
                                        dtype=conf.dtype,
                                        collections=['em_accumulators'])
                # Creates a named tuple of node, accum and TF scope
                param_node = EMLearning.ParamNode(node=node, accum=accum,
                                                  name_scope=scope)
                self._param_nodes.append(param_node)

    self._param_nodes = []
    with tf.name_scope(self._name_scope):
        # Start at root, applying fun at each position
        traverse_graph(self._root, fun=fun)

```

The key thing is to add it to the `_param_nodes` list. So let's see what `traverse_graph` does:
```python
def traverse_graph(root, fun, skip_params=False):
    """Runs ``fun`` on descendants of ``root`` (including ``root``) by
    traversing the graph breadth-first until ``fun`` returns True.

    Args:
        root (Node): The root of the SPN graph.
        fun (function): A function ``fun(node)`` executed once for every node of
                        the graph. It should return ``True`` if traversing
                        should be stopped.
        skip_params (bool): If ``True``, the param nodes will not be traversed.

    Returns:
        Node: Returns the last traversed node (the one for which ``fun``
        returned True) or ``None`` if ``fun`` never returned ``True``.
    """
    visited_nodes = set()  # Set of visited nodes
    queue = deque()
    queue.append(root)

    """ 
    In the example we see here, we eventually return None, 
    so we don't have a case where fun returns True.
    """
    while queue:
        next_node = queue.popleft()
        if next_node not in visited_nodes:
            if fun(next_node):
                return next_node
            visited_nodes.add(next_node)
            # OpNode?: enqueue inputs
            if next_node.is_op:
                """ makes sure fun() is applied to these. Optionally ignores param Node """
                for i in next_node.inputs:
                    if (i and  # Input not empty
                            not (skip_params and i.is_param)):
                        queue.append(i.node)

    return None
```

Then, we use `accumulate_updates` to build the TF graph that makes the EM updates. This is defined in the `EMLearning` class:
```python
def accumulate_updates(self):
    # Generate path if not yet generated
    if not self._mpe_path.counts:
        self._mpe_path.get_mpe_path(self._root)

    # Generate all accumulate operations
    with tf.name_scope(self._name_scope):
        assign_ops = []
        # For each of the parameter nodes that we collected in accumulator creation
        for pn in self._param_nodes:
            with tf.name_scope(pn.name_scope):
                # Get count tensor corresponding to this node
                counts = self._mpe_path.counts[pn.node]
                # Compute the update value using the counts
                update_value = pn.node._compute_hard_em_update(counts)
                # Assign this EM value to the accumulator 
                op = tf.assign_add(pn.accum, update_value)
                assign_ops.append(op)
        return tf.group(*assign_ops, name="accumulate_updates")
```
Interesting to know is the `_compute_hard_em_update` function. It is trivial:
```python
def _compute_hard_em_update(self, counts):
    # Summing over 0th axis, counting the occurences of this node
    return tf.reduce_sum(counts, axis=0)
```
More sophisticated is the function that determines the path:
```python
def get_mpe_path(self, root):
    """Assemble TF operations computing the branch counts for the MPE
    downward path through the SPN rooted in ``root``.

    Args:
        root (Node): The root node of the SPN graph.
    """
    def down_fun(node, parent_vals):
        # Sum up all parent vals
        if len(parent_vals) > 1:
            summed = tf.add_n(parent_vals, name=node.name + "_add")
        else:
            summed = parent_vals[0]
        self._counts[node] = summed # For root, this will be just be ones
        if node.is_op:
            # Compute for inputs
            with tf.name_scope(node.name):
                if self._log:
                    return node._compute_log_mpe_path(
                        summed, *[self._value.values[i.node]
                                  if i else None
                                  for i in node.inputs],
                        add_random=self._add_random,
                        use_unweighted=self._use_unweighted)
                else:
                    return node._compute_mpe_path(
                        summed, *[self._value.values[i.node]
                                  if i else None
                                  for i in node.inputs],
                        add_random=self._add_random,
                        use_unweighted=self._use_unweighted)

    # Generate values if not yet generated
    if not self._value.values:
        self._value.get_value(root)

    with tf.name_scope("MPEPath"):
        # Compute the tensor to feed to the root node
        # Basically sets input of root to ones
        graph_input = tf.ones_like(self._value.values[root])

        # Traverse the graph computing counts
        self._counts = {}
        compute_graph_up_down(root, down_fun=down_fun, graph_input=graph_input)
```
So this constructs the upward pass and the downward pass using `down_fun`. The downward function applies `_compute_log_mpe_path` to all of the nodes on the downward pass.

```python
def compute_graph_up_down(root, down_fun, graph_input, up_fun=None,
                          up_values=None, down_values=None):
    """Computes a values for every node in the graph moving first up and then down
    the graph. When moving up, it behaves exactly as :meth:`compute_graph_up`.
    When moving down it computes values for each input of a node based on
    values produced for inputs of parent nodes connected to this node. For this,
    it traverses the graph breadth-first from the ``root`` node to the leaf nodes.

    Args:
        root (Node): The root of the SPN graph.
        down_fun (function): A function ``down_fun(node, parent_vals)``
            producing values for each input of the ``node``. The argument
            ``parent_vals`` is a list containing the values obtained for each
            parent node input connected to this node.
        graph_input: The value passed as a single parent value to the function
            computing the values for the root node or a function which computes
            that value.
        up_fun (function): A function ``up_fun(node, *args)`` producing a
            certain value for the ``node``. For an op node, it will have
            additional arguments with values produced for the input nodes of
            ``node``. The arguments can be ``None`` if the input was empty.
        up_values (dict): A dictionary indexed by ``node`` in which values
            computed for each node during the upward pass will be stored. Can
            be set to ``None``.
        down_values (dict): A dictionary indexed by ``node`` in which values
            computed for each input of a node during the downward pass will be
            stored. Can be set to ``None``.
    """
    if down_values is None:  # Dictionary of computed values indexed by node
        down_values = {}
    queue = deque()  # Queue of nodes with computed values, but unprocessed inputs
    parents = defaultdict(list)  # Awesome, didn't know this trick

    def up_fun_parents(node, *args):
        """Run up_fun and for each node find parent node inputs having the node
        connected."""
        # For each input, add the node and input number as relevant parent node
        # input to the connected node
        if node.is_op:
            for nr, inpt in enumerate(node.inputs):
                if inpt:
                    parents[inpt.node].append((node, nr))
        # Run up_fun
        if up_fun is not None:
            return up_fun(node, *args)

    # Traverse up
    # up_values is none in this case, merely going up the graph while applying up_fun_parents
    compute_graph_up(root, val_fun=up_fun_parents, all_values=up_values)

    # Add root node
    if callable(graph_input):
        graph_input = graph_input()
    down_values[root] = down_fun(root, [graph_input])
    if root.is_op:
        queue.append(root)

    # Traverse down
    while queue:
        next_node = queue.popleft()
        children = set(i.node for i in next_node.inputs if i)
        for child in children:
            if child not in down_values:  # Not computed yet
                # Get all parent_vals
                parent_vals = []
                try:
                    for parent_node, parent_input_nr in parents[child]:
                        parent_vals.append(
                            down_values[parent_node][parent_input_nr])
                    # All parent values are available, compute value
                    down_values[child] = down_fun(child, parent_vals)
                    # Enqueue for further processing of children
                    if child.is_op:
                        queue.append(child)
                except KeyError:
                    # Not all parent values were available
                    pass
```

Let's see what `_compute_log_mpe_path` does for sum nodes:
```python
def _compute_log_mpe_path(self, counts, weight_value, ivs_value, *value_values,
                          add_random=None, use_unweighted=False):
    # Get weighted, IV selected values
    weight_value, ivs_value, values = self._compute_value_common(
        weight_value, ivs_value, *value_values)
    values_selected = values + ivs_value if self._ivs else values

    # WARN USING UNWEIGHTED VALUE
    if not use_unweighted or any(v.node.is_var for v in self._values):
        values_weighted = values_selected + weight_value """ multiplication <=> addition in log space """
    else:
        values_weighted = values_selected

    # / USING UNWEIGHTED VALUE

    # WARN ADDING RANDOM NUMBERS
    if add_random is not None:
        values_weighted = tf.add(values_weighted, tf.random_uniform(
            shape=(tf.shape(values_weighted)[0],
                   int(values_weighted.get_shape()[1])),
            minval=0, maxval=add_random,
            dtype=conf.dtype))
    # /ADDING RANDOM NUMBERS

    return self._compute_mpe_path_common(
        values_weighted, counts, weight_value, ivs_value, *value_values)

```

Which refers to `_compute_mpe_path_common`:
```python

def _compute_mpe_path_common(self, values_weighted, counts, weight_value,
                             ivs_value, *value_values):
    # Propagate the counts to the max value
    # Get indices of max elements
    max_indices = tf.argmax(values_weighted, dimension=1)
    # Create one_hot vector with max indices set to 1, multiplied with the counts tensor 
    max_counts = tf.one_hot(max_indices,
                            values_weighted.get_shape()[1]) * counts
    # Split the counts to value inputs
    """ Apparently, the first two elements of self.inputs are different """
    _, _, *value_sizes = self.get_input_sizes(None, None, *value_values)
    # Splits the tensor so that each of the elements can be given to the children
    # Of course, this means that we split along the 1st axis
    max_counts_split = utils.split_maybe(max_counts, value_sizes, 1)
    # Use the custom scatter op
    return self._scatter_to_input_tensors(
        (max_counts, weight_value),  # Weights
        (max_counts, ivs_value),  # IVs
        *[(t, v) for t, v in zip(max_counts_split, value_values)])  # Values
```
Which refers to `_scatter_to_input_tensors`:

```python

```

### Run Learning

In [7]:
epoch = 0
with spn.session() as (sess, run):
    sess.run(init_weights)
    sess.run(init_learning)
    while run():
        likelihood_arr, _ = sess.run([likelihood, accumulate_updates],
                                  feed_dict={iv_x:iv_x_arr, iv_y:iv_y_arr})
        print("Avg. Likelihood: %s" % (likelihood_arr))
        sess.run(update_spn)
        epoch+=1
        if epoch > 10:
            break

Avg. Likelihood: -1.2939459
Avg. Likelihood: -1.2843871
Avg. Likelihood: -1.2565484
Avg. Likelihood: -1.2453744
Avg. Likelihood: -1.2396214
Avg. Likelihood: -1.2361908
Avg. Likelihood: -1.23394
Avg. Likelihood: -1.2323612
Avg. Likelihood: -1.2311978
Avg. Likelihood: -1.2303077
Avg. Likelihood: -1.2296064


In [8]:
spn.display_tf_graph()

So what do we get when we run `spn.session()`? A TF session and a `run` function.

In [7]:
with spn.session() as (sess, run):
    sess.run(init_weights)
    sess.run(init_learning)
    try:
        while run():
            likelihoods, _ =  sess.run([likelihood, accumulate_updates])
            print("Avg. Likelihood: %s" % (avg_likelihood))
            sess.run(update_spn)
            
    except tf.errors.OutOfRangeError:
        print("TRAINING DONE!")

InvalidArgumentError: Shape [-1,2] has negative dimensions
	 [[Node: iv_x/Placeholder = Placeholder[dtype=DT_INT32, shape=[?,2], _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'iv_x/Placeholder', defined at:
  File "/home/jos/anaconda3/envs/libspn/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/jos/anaconda3/envs/libspn/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/jos/.local/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/jos/anaconda3/envs/libspn/lib/python3.5/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/jos/.local/lib/python3.5/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/jos/.local/lib/python3.5/site-packages/zmq/eventloop/ioloop.py", line 177, in start
    super(ZMQIOLoop, self).